## Error Analysis

We use the finegrained task  here



In [1]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets
import json

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Obs

`text` hace referencia al contexto del tweet del diario
`title` al título de la noticia (extraído del artículo)

In [2]:


evaluations = {
}
for model_name, path in [
    ('none-ft', "../evaluations/betonews_fine_none.json"),   
    ('text-ft', "../evaluations/betonews_fine_text.json"),   
    ('body-ft', "../evaluations/betonews_fine_body.json"),   
    ]:

    with open(path) as f:
        evaluations[model_name] = json.load(f)

for key, evals in evaluations.items():
    print(f"We have {len(evals['metrics'])} {key} evaluations")


We have 10 none-ft evaluations
We have 10 text-ft evaluations
We have 15 body-ft evaluations


In [12]:
import numpy as np
from hatedetection import extended_hate_categories

df = test_dataset.to_pandas().set_index("id")
df["HATEFUL"] = df[extended_hate_categories].any(1)

    



context_column_names = [x+"_context" for x in extended_hate_categories]
none_column_names = [x+"_none" for x in extended_hate_categories]

df[context_column_names] = (np.array(evaluations["text-ft"]["predictions"]) > 0).mean(0)
df["HATEFUL_context"] = (df[context_column_names] > 0.5).any(1)

df[none_column_names] = (np.array(evaluations["none-ft"]["predictions"]) > 0).mean(0)
df["HATEFUL_none"] = (df[none_column_names] > 0.5).any(1)

"""
Obtain offended chars for each row
"""
def get_chars(row):    
    return [k.split("_")[0] for k, v in row.items() if v > 0.5]

df["CHARS_none"] = df[none_column_names].apply(get_chars, axis=1)
df["CHARS_context"] = df[context_column_names].apply(get_chars, axis=1)

df.describe()

,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL,CALLS_context,...,CRIMINAL_context,CALLS_none,WOMEN_none,LGBTI_none,RACISM_none,CLASS_none,POLITICS_none,DISABLED_none,APPEARANCE_none,CRIMINAL_none
count,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,...,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000,11343.000000
mean,0.024861,0.023980,0.017191,0.042758,0.013312,0.025655,0.011373,0.033413,0.031914,0.020753,...,0.026148,0.020709,0.013841,0.008437,0.039954,0.010853,0.019527,0.010756,0.032099,0.023239
std,0.155709,0.152992,0.129989,0.202319,0.114613,0.158110,0.106039,0.179720,0.175779,0.133412,...,0.150197,0.132521,0.100109,0.079997,0.181947,0.090870,0.127778,0.095530,0.167408,0.136992
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## CRIMINAL

In [16]:
import pandas as pd

cat = "CRIMINAL"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240
df[df[cat] & df["HATEFUL_context"] & ~df["HATEFUL_none"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_context"]]

,article_text,text,CRIMINAL_context,CRIMINAL_none,CHARS_context
id,,,,,
358021,El polémico cortejo del ladrón asesinado por el jubilado en Quilmes url url,@usuario Le hubieran puesto una bomba al cortejo,1.0,0.0,"[CALLS, CRIMINAL]"
358068,El polémico cortejo del ladrón asesinado por el jubilado en Quilmes url url,@usuario A nadie se le ocurrió poner una bomba ahí?,1.0,0.0,"[CALLS, CRIMINAL]"
404726,"Usurpación en Cariló. Cumbia y champagne, así festejaban en la casa tomada url url",@usuario UNO NO SABE SI REIR O LLORAR... QUE GRASA MAS GRASA... A LOS TIROS HAY QUE SACARLOS,0.7,0.2,[CRIMINAL]
405653,Así quedó la iglesia de El Bolsón vandalizada por mapuches url,@usuario BASTA DE ESTOS SINVERGÜENZAS...ESPERO QUE LOS METAN TODOS PRESOS Y LE SAQUEN LOS PLANES SOCIALES!!!,0.0,0.1,"[CALLS, RACISM, CLASS]"
407351,El asesinato del ciudadano armenio reavivó la discusión sobre la imputabilidad de menores url,@usuario Que discusión? Bala,1.0,0.3,"[CALLS, CRIMINAL]"
364818,Una madre tuvo que esconder a su hija porque su vecino violador con prisión domiciliaria la acosa constantemente | Por Miguel Prieto Toledo url,@usuario Pegale un tiro a ese emoji anciano tono de piel oscuro medio emoji hdrp emoji cara con símbolos en la boca emoji emoji cara con símbolos en la boca emoji emoji cara con símbolos en la boca emoji el padre de la nena un tio alg...,0.9,0.2,[CRIMINAL]
370382,Una mujer policía de la Ciudad mató a balazos a un delincuente que robaba ruedas de autos url,@usuario Denle un aplauso a esa buena mujer ! emoji manos aplaudiendo emoji emoji manos aplaudiendo emoji emoji manos aplaudiendo emoji,1.0,0.0,[CRIMINAL]
370383,Una mujer policía de la Ciudad mató a balazos a un delincuente que robaba ruedas de autos url,"@usuario La felicito ,es más le doy un ascenso ya",0.7,0.0,[CRIMINAL]
370391,Una mujer policía de la Ciudad mató a balazos a un delincuente que robaba ruedas de autos url,@usuario Genia! La mujeres q todos qremos.,0.7,0.0,[CRIMINAL]


| Contexto                         |           Texto                                                                                         |
|----------------------------------|---------------------------------------------------------------------------------------------------------|
| Una policía baleó y mató a un joven de 17 años que la atacó con una tijera en Moreno url	| bellezaaa                                      |
|                                                                                           | 👏 excelente!                                  |
| El polémico cortejo del ladrón asesinado por el jubilado en Quilmes | Le hubieran puesto una bomba al cortejo                              |


## LGBTI


In [17]:
import pandas as pd

cat = "LGBTI"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240

df[df[cat] & df["HATEFUL_context"] & ~df["HATEFUL_none"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_context"]]

,article_text,text,LGBTI_context,LGBTI_none,CHARS_context
id,,,,,
375847,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nurl","@usuario Revisen esa casa, los están envenenando.",0.6,0.0,[LGBTI]
375859,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nurl","@usuario El hermano se coge al perro, no?",0.6,0.0,[LGBTI]
375872,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nurl",@usuario emoji cara vomitando emoji emoji cara vomitando emoji emoji cara vomitando emoji,0.9,0.0,[LGBTI]
398893,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,@usuario Igual deberá visitar al médico para revisión de próstata emoji cara pensativa emoji,0.6,0.0,[LGBTI]
398905,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,"@usuario Bien Roberto, lo lograste: en un equipo de tipos no tenias chancho, pero le buscaste la vuelta y llegaste... Pais de emprendedores! emoji cara sonriendo con los ojos cerrados emoji",0.6,0.0,[LGBTI]
398912,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,@usuario Tiene más frente de la Casa Blanca \nEsta quedando pelado el guacho,0.6,0.0,[LGBTI]
398917,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,@usuario A romper piernas muchacho,1.0,0.0,[LGBTI]
398920,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,"@usuario Mara ""garrote"" Gómez",0.8,0.0,[LGBTI]
398925,Mara Gómez: la historia de la primera futbolista trans en el torneo argentino url url,"@usuario Sigue siendo HOMBRE, que por GENÉTICA, no por una ideología u orientacion sexual, GE-NÉ-TI-CA, es más fuerte que la mujer.",1.0,0.3,[LGBTI]


| Contexto                         |           Texto                                                                                         |
|----------------------------------|---------------------------------------------------------------------------------------------------------|
|"Te amo ". La emotiva dedicatoria de Luis Novaresio a su pareja en su cumpleaños | Definitivamente no acepto esta degeneración repugnante de la humanidad. |
| Mara Gómez: la historia de la primera futbolista trans en el torneo argentino | Sigue siendo HOMBRE, que por GENÉTICA, no por una ideología u orientacion sexual, GE-NÉ-TI-CA, es más fuerte que la mujer. |
| Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film| Revisen esa casa, los están envenenando.|


## CLASS

In [18]:
import pandas as pd

cat = "CLASS"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240

df[df[cat] & df["HATEFUL_context"] & ~df["HATEFUL_none"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_context"]]

,article_text,text,CLASS_context,CLASS_none,CHARS_context
id,,,,,
398859,"Empujones, corridas y botellazos en la despedida a Diego Maradona url",@usuario soltás a los negritos 2 minutos y se matan.\no bien festejan de la única forma que saben\ndespués el criminal es el que pide abrir las escuelas.,0.0,0.1,[RACISM]
343209,"En medio de la cuarentena por el coronavirus, el Gobierno avanza con un impuesto a los abonos de Netflix url",@usuario ESTE TÍTERE ES LO MÁS HDRMP...TODAVÍA QUE MANTENEMOS A SUS PARÁSITOS K Y PLANEROS QUIERE SACARNOS MÁS PLATA...\nTITERE CON YEGUA LINDA JUNTA!!!,0.4,0.1,[POLITICS]
334366,"Juan Grabois: ""Hay que aplicar la autoridad sobre las personas que se manifestaron"" url",@usuario Y bala para los que toman tierras ajenas y para los que lo promueven !! Cuidado grabois ahí va una !!!,0.9,0.5,"[CALLS, CLASS, POLITICS]"
405653,Así quedó la iglesia de El Bolsón vandalizada por mapuches url,@usuario BASTA DE ESTOS SINVERGÜENZAS...ESPERO QUE LOS METAN TODOS PRESOS Y LE SAQUEN LOS PLANES SOCIALES!!!,0.8,0.5,"[CALLS, RACISM, CLASS]"
405013,Otro día de marchas y cortes en el centro porteño: 3 caravanas se movilizan en apoyo al Gobierno url,"@usuario Vagos, lacras!!!",1.0,0.1,[CLASS]
357690,"El mensaje de El Dipy que lo convirtió en tendencia en las redes: ""Piensan que porque salí de un barrio humilde, soy kirchnerista o soy peronista"" url","@usuario Jjaja, no entras en ninguna categoría papá!!! Ahhh y hacete bien esos dientes dracu dracu que tenes! Otario",0.0,0.0,[APPEARANCE]
392268,"Coronavirus: Alberto Fernández advirtió que, 'si el relajamiento sigue, las fuerzas de seguridad saldrán a la calle para disipar a la gente' url",@usuario emoji cara revolviéndose de la risa emoji emoji cara revolviéndose de la risa emoji emoji cara revolviéndose de la risa emoji siii hacerme reor payaso esto l9 hiciste vos d3l8ncuente con un velorio y piquete de los parasitos...,0.9,0.5,"[CLASS, POLITICS]"
401976,La temporada de verano en la Costa Atlántica empezó con un corte total en la Ruta 2: organizaciones sociales piden canastas navideñas url,@usuario @usuario Jjaja emoji cara con mano sobre la boca emoji emoji cara revolviéndose de la risa emoji emoji cara revolviéndose de la risa emoji emoji cara revolviéndose de la risa emoji siempre los vividores,0.8,0.2,[CLASS]
401977,La temporada de verano en la Costa Atlántica empezó con un corte total en la Ruta 2: organizaciones sociales piden canastas navideñas url,@usuario Pais bananero LABUREN lpm,0.9,0.3,[CLASS]


| Contexto                         |           Texto                                                                                         |
|----------------------------------|---------------------------------------------------------------------------------------------------------|
| Las organizaciones sociales salieron al cruce de la acusación de Sergio Berni por la toma de tierras: "La falta de vivienda no se resuelve con balas" | Si, los podemos cagar matando a todos y listo	|
|                                       | @usuario Dale alas y planes a los cuervos y veras cómo te sacan los ojos |
| La temporada de verano en la Costa Atlántica empezó con un corte total en la Ruta 2: organizaciones sociales piden canastas navideñas | Hay que desparasitar urgente el país.	



## RACISM

In [22]:
import pandas as pd

cat = "RACISM"

pd.options.display.max_rows = None
pd.options.display.max_colwidth = 240

df[df[cat] & df["HATEFUL_context"] & ~df["HATEFUL_none"] ].loc[:, ["article_text", "text", cat+"_context", cat+"_none", "CHARS_context"]].sort_values("RACISM_context", ascending=False)

,article_text,text,RACISM_context,RACISM_none,CHARS_context
id,,,,,
393060,China lanzó una peligrosa venganza por los reportes sobre la eficacia de una de sus vacunas url,@usuario El pais mas mugriento y mentiroso que podemos esperar de esa gente que solo le importa prevalecer ante todo.,1.0,0.2,[RACISM]
405653,Así quedó la iglesia de El Bolsón vandalizada por mapuches url,@usuario BASTA DE ESTOS SINVERGÜENZAS...ESPERO QUE LOS METAN TODOS PRESOS Y LE SAQUEN LOS PLANES SOCIALES!!!,1.0,0.0,"[CALLS, RACISM, CLASS]"
393815,"Un refugio para animales en China habría vendido pieles de tigres en peligro de extinción y hecho un vino ""afrodisíaco"" con sus huesos url",@usuario COSTUMBRES Y CULTURAS CHINAS SON REPUDIABLES Y PARA NUNCA IMITAR.,1.0,0.5,[RACISM]
393808,"Un refugio para animales en China habría vendido pieles de tigres en peligro de extinción y hecho un vino ""afrodisíaco"" con sus huesos url",@usuario Cbinos de mierda! No hace 1 como gente!,1.0,0.2,[RACISM]
403672,Tensión en Villa Mascardi: mapuches resisten el desalojo de un predio usurpado url,@usuario Patada en el ogt a los ladrones !,1.0,0.0,"[CALLS, RACISM]"
403633,Tensión en Villa Mascardi: mapuches resisten el desalojo de un predio usurpado url,"@usuario Con el mismo dialogo podriamos decir que las malvinas realmente son inglesas porque un ancestro se los dijo, no? HAY QUE SACARLOS A TIROS A ESTOS HIJOS DE PUTA",1.0,0.5,"[CALLS, RACISM]"
354155,Alarma en China: más de 3000 personas se enfermaron tras la fuga de una bacteria url url,@usuario Suspendan china,1.0,0.3,"[CALLS, RACISM]"
354171,Alarma en China: más de 3000 personas se enfermaron tras la fuga de una bacteria url url,@usuario Brutos asesinos,1.0,0.0,[RACISM]
364665,Alertan por un nuevo virus en China con potencial de pandemia\nlo tienen los cerdos similar ala gripe porcina pero peor pará un poco2020 una pandemia es demasiado dos no se aguantan china lo hizo de nuevo yahora qué? estamos perdirijill...,@usuario @usuario Lo que sucede es que todos los CHINOS SON MUGRIENTOS.,1.0,0.4,[RACISM]


| Contexto                         |           Texto                                                                                         |
|----------------------------------|---------------------------------------------------------------------------------------------------------|
| La pregunta billonaria: ¿China debería pagar el costo de la pandemia? | Si obviamente y desaparecer de la faz de la tierra. Mira el quilombo que armó. Se nos están muriendo todos...	|
| China: Vuelven a vender murciélagos y carne de animales domésticos a pesar del Coronavirus |  @usuario EEUU haceme limpieza ahí.	|
|Tensión en Villa Mascardi: mapuches resisten el desalojo de un predio usurpado | Era mas facil si usaban napalm |


